In [1]:
# import linecache
# from collections import Counter
# import os
# import tracemalloc
#
# def display_top(snapshot, key_type='lineno', limit=3):
#     snapshot = snapshot.filter_traces((
#         tracemalloc.Filter(False, "<frozen importlib._bootstrap>"),
#         tracemalloc.Filter(False, "<unknown>"),
#     ))
#     top_stats = snapshot.statistics(key_type)
#
#     print("Top %s lines" % limit)
#     for index, stat in enumerate(top_stats[:limit], 1):
#         frame = stat.traceback[0]
#         # replace "/path/to/module/file.py" with "module/file.py"
#         filename = os.sep.join(frame.filename.split(os.sep)[-2:])
#         print("#%s: %s:%s: %.1f KiB"
#               % (index, filename, frame.lineno, stat.size / 1024))
#         line = linecache.getline(frame.filename, frame.lineno).strip()
#         if line:
#             print('    %s' % line)
#
#     other = top_stats[limit:]
#     if other:
#         size = sum(stat.size for stat in other)
#         print("%s other: %.1f KiB" % (len(other), size / 1024))
#     total = sum(stat.size for stat in top_stats)
#     print("Total allocated size: %.1f KiB" % (total / 1024))

# tracemalloc.start()
# counts = Counter()
# experimentation(classification_dataset_names[0])
# snapshot = tracemalloc.take_snapshot()
# display_top(snapshot)

In [2]:
import numpy as np

from ex_func import *
from experiment_functions import *
import pandas as pd
from pmlb import fetch_data, classification_dataset_names, regression_dataset_names
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectKBest, f_classif
import pickle

9ed379f63a363f5e759aa0e824692d130d8d8ad83880517bac52e890496b43cf


In [3]:
def get_feature_type(x, include_binary=False):
    x.dropna(inplace=True)
    if not check_if_all_integers(x):
        return 'continuous'
    else:
        if x.nunique() > 10:
            return 'continuous'
        if include_binary:
            if x.nunique() == 2:
                return 'binary'
        return 'categorical'

def get_target_type(x, include_binary=False):
    x.dropna(inplace=True)
    if x.dtype=='float64':
        return 'continuous'
    elif x.dtype=='int64':
        if include_binary:
            if x.nunique() == 2:
                return 'binary'
        return 'categorical'
    else:
        raise ValueError("Error getting type")

def check_if_all_integers(x):
    "check a pandas.Series is made of all integers."
    return all(float(i).is_integer() for i in x.unique())
def corr_data_for(df):
    TARGET_NAME = 'target'
    feat_names = [col for col in df.columns if col!=TARGET_NAME]
    types = [get_feature_type(df[col], include_binary=True) for col in feat_names]
    col = pd.DataFrame(feat_names,types)
    num_col = col[col.index == 'continuous']
    bin_col = col[col.index == 'binary']
    cat_col = col[col.index == 'categorical']
    cat_col = cat_col[0].tolist()
    dummy_col = pd.get_dummies(data=df, columns=cat_col)
    add_col = dummy_col.shape[1] - df.shape[1]

    if (add_col < df.shape[0] *0.3) & (dummy_col.shape[1] <  df.shape[0]) & (df.shape[0] < 10000) & (df.shape[1] < 100):
        df = dummy_col
        df.columns = df.columns.str.replace('.','_',regex=True)
        y = df['target']
        X = df.loc[:, df.columns != 'target']
        del df
        rows_data, columns_data = X.shape
        print('Dataset Information')
        print('Rows:',rows_data,)
        print('Columns:',columns_data)
        print('Number of classes:',y.nunique())
        print('Continous columns:', len(num_col))
        print('Binary columns:', len(bin_col))
        print('Categorical columns:',len(cat_col))
        print('-------------------------------------------------')
    else:
        del df
        return pd.DataFrame, pd.DataFrame
    return y, X

In [4]:
def split_function(y,X,it):
    sc = StandardScaler()
    X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.2, random_state = it
                                                        #,stratify=y
                                                        )
    X_col = X_train.columns
    X_test.name = "X_test"
    X_train.name = "X_train"
    X_train = sc.fit_transform(X_train)
    X_test = sc.transform(X_test)
    X_train = pd.DataFrame(X_train,columns=X_col)
    X_test = pd.DataFrame(X_test,columns=X_col)
    return X_train, X_test, y_train, y_test

In [5]:
# regression_dataset_names = list(np.setdiff1d(regression_dataset_names, ['527_analcatdata_election2000', '542_pollution', '659_sleuth_ex1714',        '678_visualizing_environmental', '192_vineyard', '522_pm10',        '665_sleuth_case2002', 'titanic']))

In [6]:
def experimentation(classification_dataset):
    res_rul = {}
    rules_alg = []
    coefficients = []
    names = ['Reg-CART','CART','ORT','OCT','ORT-H','OCT-H','ORT+ORT-H','OCT+OCT-H']
    algorithms = ['LR','LS','RG']
    pipelines = [LR_pipeline]

    iters = 1
    df = fetch_data(classification_dataset)
    df.drop(['Density','Abdomen'],inplace=True,axis=1)
    print('Numer of NANs: ',df.isna().sum().sum())
    y, X = corr_data_for(df)

    del df
    if X.empty:
        return {}
    if len(set(y)) < 50 | ('fri' in classification_dataset):
        return {}


    print(color.BOLD + '\n\n    ----------------------------------------- {} -----------------------------------------'.format(classification_dataset) + color.END)
    for it in range(iters):
        X_train, X_test, y_train, y_test = split_function(y,X,it)
        col_len = len(X_train.columns)
        factors = [0.5,1,1.2,1.4,1.6,1.8,2,2.5,3]

        models, performance = generate_tree(X_train, y_train, X_test, y_test, n_num=1, feat_size=len(X.columns),  max_iter_hy=2,sub_paths=True,depth_grid=range(3,4), depth_grid_hy=range(1,2), complexity_bi = 0.0001, complexity_hy=0.0001,  Reg_CART=True, ORT=True, ORT_H=False, Clas_CART=False, OCT=False, OCT_H=False)

        for perf,name in zip(performance,names):
            if not not perf:
                res_tree = sum(perf) / len(perf)
                res_rul[(classification_dataset,name,it,1)] = res_tree
                res_rul[(classification_dataset,name,it,'all')] = res_tree

        act_name = []
        act_rules = []
        for model,name in zip(models,names):
            if (all(model)) & (not not model) & (None not in model):
                act_name.append(name)
                act_rules.append(model)
        datasets = gen_train_and_test_features(act_rules ,act_name , X_train, X_test)

        for model in datasets.keys():
            X_train_rules_and_features, X_test_rules_and_features = datasets[model][0]
            rules_alg.append(X_train_rules_and_features.columns)
            X_train_only_rules, X_test_only_rules = datasets[model][1]

            for algorithm,pipeline in zip(algorithms,pipelines):
                pip_result_only_rules, coef = pipeline(X_train_only_rules, X_test_only_rules, y_train, y_test)
                pip_result_rules_features, coef  = pipeline(X_train_rules_and_features, X_test_rules_and_features, y_train, y_test)
                coefficients.append(coef)
                res_rul[(classification_dataset,model + f'_{algorithm}_rules',it,'all')] = pip_result_only_rules
                res_rul[(classification_dataset,model + f'_{algorithm}_rules_and_features',it,'all')] = pip_result_rules_features

            for fact in factors:
                if (round(len(X_train_rules_and_features.columns)*fact) <= X_train.shape[0]) & (round(col_len*fact) <= len(X_train_rules_and_features.columns)):
                    min_feat_rule = round(col_len*fact)

                    if (round(col_len*fact) > len(X_train_only_rules.columns)) & (fact != 0.5):
                        len_rule = 1
                        min_rule = len(X_train_only_rules.columns)
                    else:
                        len_rule = fact
                        min_rule = min(round(col_len*fact),len(X_train_only_rules.columns))

                    cols = SelectKBest(k=min_feat_rule).fit(X_train_rules_and_features,y_train).get_feature_names_out()
                    cols_rule = SelectKBest(k=min_rule).fit(X_train_only_rules,y_train).get_feature_names_out()

                else:
                     continue

                for algorithm,pipeline in zip(algorithms,pipelines):
                    pip_result_only_rules, coef = pipeline(X_train_only_rules[cols_rule], X_test_only_rules[cols_rule], y_train, y_test)
                    pip_result_rules_features, coef = pipeline(X_train_rules_and_features[cols], X_test_rules_and_features[cols], y_train, y_test)
                    res_rul[(classification_dataset,model + f'_{algorithm}_rules',it,len_rule)] = pip_result_only_rules
                    res_rul[(classification_dataset,model + f'_{algorithm}_rules_and_features',it,fact)] = pip_result_rules_features

        for algorithm,pipeline in zip(algorithms,pipelines):
            pip_result = pipeline(X_train, X_test, y_train, y_test)
            res_rul[(classification_dataset,algorithm,it,1)] = pip_result
            res_rul[(classification_dataset,algorithm,it,'all')] = pip_result
        del X_train, X_test

    return res_rul, rules_alg, coefficients

In [7]:
regression_dataset_names

['1027_ESL',
 '1028_SWD',
 '1029_LEV',
 '1030_ERA',
 '1089_USCrime',
 '1096_FacultySalaries',
 '1191_BNG_pbc',
 '1193_BNG_lowbwt',
 '1196_BNG_pharynx',
 '1199_BNG_echoMonths',
 '1201_BNG_breastTumor',
 '1203_BNG_pwLinear',
 '1595_poker',
 '192_vineyard',
 '195_auto_price',
 '197_cpu_act',
 '201_pol',
 '207_autoPrice',
 '210_cloud',
 '215_2dplanes',
 '218_house_8L',
 '225_puma8NH',
 '227_cpu_small',
 '228_elusage',
 '229_pwLinear',
 '230_machine_cpu',
 '294_satellite_image',
 '344_mv',
 '4544_GeographicalOriginalofMusic',
 '485_analcatdata_vehicle',
 '503_wind',
 '505_tecator',
 '519_vinnie',
 '522_pm10',
 '523_analcatdata_neavote',
 '527_analcatdata_election2000',
 '529_pollen',
 '537_houses',
 '542_pollution',
 '547_no2',
 '556_analcatdata_apnea2',
 '557_analcatdata_apnea1',
 '560_bodyfat',
 '561_cpu',
 '562_cpu_small',
 '564_fried',
 '573_cpu_act',
 '574_house_16H',
 '579_fri_c0_250_5',
 '581_fri_c3_500_25',
 '582_fri_c1_500_25',
 '583_fri_c1_1000_50',
 '584_fri_c4_500_25',
 '586_fri

In [8]:
classification_dataset_names = ['560_bodyfat']

In [83]:
classification_dataset_names

['560_bodyfat']

In [84]:
data = fetch_data('560_bodyfat')

In [90]:
data

,Density,Age,Weight,Height,Neck,Chest,Abdomen,Hip,Thigh,Knee,Ankle,Biceps,Forearm,Wrist,target
0,1.0708,23.0,154.25,67.75,36.200001,93.099998,85.199997,94.500000,59.000000,37.299999,21.900000,32.000000,27.400000,17.100000,12.300000
1,1.0853,22.0,173.25,72.25,38.500000,93.599998,83.000000,98.699997,58.700001,37.299999,23.400000,30.500000,28.900000,18.200001,6.100000
2,1.0414,22.0,154.00,66.25,34.000000,95.800003,87.900002,99.199997,59.599998,38.900002,24.000000,28.799999,25.200001,16.600000,25.299999
3,1.0751,26.0,184.75,72.25,37.400002,101.800003,86.400002,101.199997,60.099998,37.299999,22.799999,32.400002,29.400000,18.200001,10.400000
4,1.0340,24.0,184.25,71.25,34.400002,97.300003,100.000000,101.900002,63.200001,42.200001,24.000000,32.200001,27.700001,17.700001,28.700001
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
247,1.0736,70.0,134.25,67.00,34.900002,89.199997,83.599998,88.800003,49.599998,34.799999,21.500000,25.600000,25.700001,18.500000,11.000000
248,1.0236,72.0,201.00,69.75,40.900002,108.500000,105.000000,104.500000,59.599998,40.799999,23.200001,35.200001,28.600000,20.100000,33.599998
249,1.0328,72.0,186.75,66.00,38.900002,111.099998,111.500000,101.699997,60.299999,37.299999,21.500000,31.299999,27.200001,18.000000,29.299999
250,1.0399,72.0,190.75,70.50,38.900002,108.300003,101.300003,97.800003,56.000000,41.599998,22.700001,30.500000,29.400000,19.799999,26.000000


In [9]:
# for data in classification_dataset_names:
#     data = fetch_data(data)
#     print(data.shape)

In [10]:
orig = {}
rules = []
for classification_dataset in classification_dataset_names:
    res_rul,rules,coefficients = experimentation(classification_dataset)
    rules.append(rules)
    orig.update(res_rul)

Numer of NANs:  0
Dataset Information
Rows: 252
Columns: 12
Number of classes: 176
Continous columns: 12
Binary columns: 0
Categorical columns: 0
-------------------------------------------------


    ----------------------------------------- 560_bodyfat -----------------------------------------


[ Warning: This copy of Interpretable AI software is for academic purposes only and not for commercial use.
┌ Warning: Interpretable AI license expires soon: 2022-12-31T00:00:00. If you need to renew, please send us the following machine ID:
└ 9ed379f63a363f5e759aa0e824692d130d8d8ad83880517bac52e890496b43cf


Regression CART mean performance:  0.5674612739375167


Regression ORT performance:  0.5083570484134248




In [91]:
inverse_transform(X, copy=None)

,Density,Age,Weight,Height,Neck,Chest,Abdomen,Hip,Thigh,Knee,Ankle,Biceps,Forearm,Wrist,target
0,1.0708,23.0,154.25,67.75,36.200001,93.099998,85.199997,94.500000,59.000000,37.299999,21.900000,32.000000,27.400000,17.100000,12.300000
1,1.0853,22.0,173.25,72.25,38.500000,93.599998,83.000000,98.699997,58.700001,37.299999,23.400000,30.500000,28.900000,18.200001,6.100000
2,1.0414,22.0,154.00,66.25,34.000000,95.800003,87.900002,99.199997,59.599998,38.900002,24.000000,28.799999,25.200001,16.600000,25.299999
3,1.0751,26.0,184.75,72.25,37.400002,101.800003,86.400002,101.199997,60.099998,37.299999,22.799999,32.400002,29.400000,18.200001,10.400000
4,1.0340,24.0,184.25,71.25,34.400002,97.300003,100.000000,101.900002,63.200001,42.200001,24.000000,32.200001,27.700001,17.700001,28.700001
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
247,1.0736,70.0,134.25,67.00,34.900002,89.199997,83.599998,88.800003,49.599998,34.799999,21.500000,25.600000,25.700001,18.500000,11.000000
248,1.0236,72.0,201.00,69.75,40.900002,108.500000,105.000000,104.500000,59.599998,40.799999,23.200001,35.200001,28.600000,20.100000,33.599998
249,1.0328,72.0,186.75,66.00,38.900002,111.099998,111.500000,101.699997,60.299999,37.299999,21.500000,31.299999,27.200001,18.000000,29.299999
250,1.0399,72.0,190.75,70.50,38.900002,108.300003,101.300003,97.800003,56.000000,41.599998,22.700001,30.500000,29.400000,19.799999,26.000000


In [118]:
mean_weight = data['Height'].mean()
mean_weight_std = data['Height'].std()

In [119]:
0.482*mean_weight_std+mean_weight

71.91430601347143

In [115]:
183.77

183.77

In [103]:
data.describe()

,Density,Age,Weight,Height,Neck,Chest,Abdomen,Hip,Thigh,Knee,Ankle,Biceps,Forearm,Wrist,target
count,252.000000,252.000000,252.000000,252.000000,252.000000,252.000000,252.000000,252.000000,252.000000,252.000000,252.000000,252.000000,252.000000,252.000000,252.000000
mean,1.055574,44.884921,178.924405,70.148810,37.992064,100.824206,92.555952,99.904762,59.405952,38.590476,23.102381,32.273413,28.663889,18.229762,19.150794
std,0.019031,12.602040,29.389160,3.662856,2.430913,8.430476,10.783077,7.164058,5.249952,2.411804,1.694894,3.021274,2.020691,0.933585,8.368740
min,0.995000,22.000000,118.500000,29.500000,31.100000,79.300003,69.400002,85.000000,47.200001,33.000000,19.100000,24.799999,21.000000,15.800000,0.000000
25%,1.041400,35.750000,159.000000,68.250000,36.400002,94.350000,84.574999,95.500000,56.000000,36.975000,22.000000,30.200001,27.299999,17.600000,12.475000
50%,1.054900,43.000000,176.500000,70.000000,38.000000,99.649998,90.950001,99.300003,59.000000,38.500000,22.799999,32.049999,28.700001,18.299999,19.200001
75%,1.070400,54.000000,197.000000,72.250000,39.425001,105.375002,99.324997,103.525000,62.349999,39.925001,24.000000,34.325000,30.000000,18.799999,25.299999
max,1.108900,81.000000,363.149994,77.750000,51.200001,136.199997,148.100006,147.699997,87.300003,49.099998,33.900002,45.000000,34.900002,21.400000,47.500000


In [93]:
data['Height'].mean()
data['Height'].mean()

70.14880952380952

In [11]:
coefficients

[[21.088015568971308,
  array([ 2.29873927,  2.70200269, -0.65604061, -1.058245  ,  1.6474585 ,
          1.50942998,  1.48509101, -0.58331955, -0.20032325, -0.63300611,
          0.56157606, -2.32322485, -6.26982326, -3.08569504, -0.02399061,
         -4.84097242, -0.33093284,  1.51567005])],
 [24.194909519747863,
  array([ 1.84360066,  2.57889212, -0.52256208, -1.01931442,  1.62638597,
          1.15305536,  1.1928441 , -0.49192767, -0.32785958, -0.4906077 ,
          0.39817844, -1.87368126, -9.64664026, -1.92099746, -6.1685685 ,
         -0.94540549, -8.8084112 , -2.06283972])],
 [24.194909519747863,
  array([ 1.84360066,  2.57889212, -0.52256208, -1.01931442,  1.62638597,
          1.15305536,  1.1928441 , -0.49192767, -0.32785958, -0.4906077 ,
          0.39817844, -1.87368126, -9.64664026, -1.92099746, -6.1685685 ,
         -0.94540549, -8.8084112 , -2.06283972])]]

In [ ]:
'(feature['Weight'] >= 0.165) & (feature['Age'] < -0.529) & (feature['Height'] >= 0.482)'

In [12]:
rules

[Index(['Age', 'Weight', 'Height', 'Neck', 'Chest', 'Hip', 'Thigh', 'Knee',
        'Ankle', 'Biceps', 'Forearm', 'Wrist',
        '(feature['Chest'] < 0.107) & (feature['Chest'] < -0.772) & (feature['Thigh'] < -0.11)',
        '(feature['Chest'] < 0.107) & (feature['Chest'] < -0.772) & (feature['Thigh'] >= -0.11)',
        '(feature['Chest'] < 0.107) & (feature['Chest'] >= -0.772) & (feature['Height'] < 0.418)',
        '(feature['Chest'] < 0.107) & (feature['Chest'] >= -0.772) & (feature['Height'] >= 0.418)',
        '(feature['Chest'] >= 0.107) & (feature['Chest'] < 1.471) & (feature['Height'] < 1.282)',
        '(feature['Chest'] >= 0.107) & (feature['Chest'] >= 1.471) & (feature['Height'] >= -1.086)'],
       dtype='object'),
 Index(['Age', 'Weight', 'Height', 'Neck', 'Chest', 'Hip', 'Thigh', 'Knee',
        'Ankle', 'Biceps', 'Forearm', 'Wrist',
        '(feature['Weight'] < 0.165) & (feature['Chest'] < -0.743) & (feature['Thigh'] < 0.146)',
        '(feature['Weight'] < 0.165) &

In [9]:
# from joblib import delayed
# from tqdm import tqdm
# res_rul = ProgressParallel(n_jobs=8)(delayed(experimentation)(data) for data in regression_dataset_names)

0it [00:00, ?it/s]

ERROR: error initializing stdin in uv_pipe_open: inappropriate ioctl for device (ENOTTY -25)
ERROR: ERROR: error initializing stdin in uv_pipe_open: inappropriate ioctl for device (ENOTTY -25)
error initializing stdin in uv_pipe_open: inappropriate ioctl for device (ENOTTY -25)
ERROR: error initializing stdin in uv_pipe_open: inappropriate ioctl for device (ENOTTY -25)ERROR: error initializing stdin in uv_pipe_open: inappropriate ioctl for device (ENOTTY -25)

ERROR: error initializing stdin in uv_pipe_open: inappropriate ioctl for device (ENOTTY -25)
ERROR: error initializing stdin in uv_pipe_open: inappropriate ioctl for device (ENOTTY -25)
ERROR: error initializing stdin in uv_pipe_open: inappropriate ioctl for device (ENOTTY -25)


BrokenProcessPool: A task has failed to un-serialize. Please ensure that the arguments of the function are all picklable.

In [ ]:
result = {}
for d in res_rul:
    result.update(d)

In [ ]:
# import os
# files = os.listdir('C:/Users/paulr/PycharmProjects/pythonProject/ORRFA-2/')

In [ ]:
# list_of_dfs = []
# for file in files:
#     if file.endswith('pickle'):
#         with open(file, 'rb') as handle:
#             b = pickle.load(handle)
#             # df = pd.concat({k:json_normalize(v, 'scores', ['best']) for k,v in d.items()})
#             # df = df.reset_index(level=1, drop=True).rename_axis('names').reset_index()
# #             list_of_dfs.append(b)
# big_df = pd.concat(list_of_dfs, ignore_index=True)#ignore_index to reset index of big_df
# big_df.head()

In [ ]:
# k = pd.DataFrame(result,index=[0])

In [ ]:
#k.to_csv('results/results_gridsearch.csv')

In [ ]:
g

In [ ]:
import os
os.getcwd()

In [ ]:
with open('results_regression_depth4.pkl', 'wb') as f:
    pickle.dump(result, f)

In [ ]:
k = pd.DataFrame(result,index=[0])
k = k.stack(level=2).sort_index()
k = k.stack(level=2).sort_index()
k = k.swaplevel(axis=1)
k = k.droplevel(0)

In [ ]:
k.mean(level=0,axis=1)

In [ ]:
k.to_csv('results_regression_depth3.csv')

In [ ]:
# k.to_csv('result_girdsearch.csv')

In [ ]:
k.swaplevel(axis=0).mean(level=0).mean(level=0,axis=1).iloc[9].sort_values(ascending=False)

In [ ]:
k.swaplevel(axis=0).mean(level=0).mean(level=0,axis=1).iloc[1].sort_values(ascending=False)

In [ ]:
t=k.mean(level=0,axis=1)
t = t.mean(axis=0)
t.sort_values(ascending = False)

In [ ]:
y = k.swaplevel(axis=1)
y = y.var(level=0,axis=1)
y = y.mean(axis=0)
good_tests = y[y < 0.01].index
good = list(good_tests)

In [ ]:
vaild_results = k.iloc[:,k.columns.isin(good, level=1)]
vaild_results=vaild_results.mean(level=0,axis=1)
vaild_results.mean(axis=0)

In [ ]:
import matplotlib.pyplot as plt
fig, ax = plt.subplots(nrows = 5, ncols = 4, gridspec_kw = {"hspace": 0.25})
import seaborn as sns
fig.set_size_inches(30, 25)
iteration = 0

for m in range(5):
    for j in range(4):

        dataset = classification_dataset_names[:20][iteration]

        columns = [i for i in k.columns if dataset in i]
        sns.boxplot(k[columns], ax = ax[m, j])

        ax[m, j].set_title(dataset)

        ax[m, j].set_xticklabels(['CART Rules', "OCT Rules", "Logistic Regression", "RuleFit", "ORRFA"])

        iteration += 1



In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
fig, ax = plt.subplots()
sns.boxplot(data = performance_by_iter)
fig.set_size_inches(20, 10)
ax.set_xticklabels(performance_by_iter.columns.values)
# ax.set_ylim(0.93, 0.995)
ax.tick_params(rotation = 0, labelsize = 14)
ax.set_ylabel("Accuracy", fontsize = 14)
ax.set_title("Accuracy of Logistic Regression, RuleFit and ORRFA", fontsize = 15)
# ax.set_ylabel()
plt.savefig('Benchmark ORRFA.png')

In [ ]:
from pmlb import fetch_data, classification_dataset_names

In [ ]:
res_rul = ProgressParallel(n_jobs=15)(delayed(experimentation)(data) for data in regression_dataset_names)

In [ ]:
classification_dataset = regression_dataset_names[0]

In [ ]:

res_rul = {}
names = ['Reg-CART','CART','ORT','OCT','ORT-H','OCT-H','ORT+ORT-H','OCT+OCT-H']
algorithms = ['LR','LS','RG']
pipelines = [LR_pipeline,LS_pipeline,RG_pipeline]

iters = 5
df = fetch_data(classification_dataset)
print('Numer of NANs: ',df.isna().sum().sum())
y, X = corr_data_for(df)

del df

print(color.BOLD + '\n\n    ----------------------------------------- {} -----------------------------------------'.format(classification_dataset) + color.END)
for it in range(iters):
    X_train, X_test, y_train, y_test = split_function(y,X,it)
    col_len = len(X_train.columns)
    factors = [0.5,1,1.2,1.4,1.6,1.8,2,2.5,3]

    models, performance = generate_tree(X_train, y_train, X_test, y_test, n_num=1, feat_size=len(X.columns),  max_iter_hy=2,sub_paths=True,depth_grid=range(3,4), depth_grid_hy=range(1,3), complexity_bi = 0.001, complexity_hy=0.001,  Reg_CART=True, ORT=True, ORT_H=False, Clas_CART=False, OCT=False, OCT_H=False)

    for perf,name in zip(performance,names):
        if not not perf:
            res_rul[(classification_dataset,name,it,1)] = sum(perf) / len(perf)

    act_name = []
    act_rules = []
    for model,name in zip(models,names):
        if (all(model)) & (not not model) & (None not in model):
            act_name.append(name)
            act_rules.append(model)

    datasets = gen_train_and_test_features(act_rules ,act_name , X_train, X_test)

    for model in datasets.keys():
        X_train_rules_and_features, X_test_rules_and_features = datasets[model][0]
        X_train_only_rules, X_test_only_rules = datasets[model][1]

        for algorithm,pipeline in zip(algorithms,pipelines):
            res_rul[(classification_dataset,model + f'_{algorithm}_rules',it,'all')] = pipeline(X_train_only_rules, X_test_only_rules, y_train, y_test)
            res_rul[(classification_dataset,model + f'_{algorithm}_rules_and_features',it,'all')] = pipeline(X_train_rules_and_features, X_test_rules_and_features, y_train, y_test)

        for fact in factors:
            if (round(len(X_train_rules_and_features.columns)*fact) <= X_train.shape[0]) & (round(col_len*fact) <= len(X_train_rules_and_features.columns)):
                min_feat_rule = round(col_len*fact)

                if (round(col_len*fact) > len(X_train_only_rules.columns)) & (fact != 0.5):
                    len_rule = 1
                    min_rule = len(X_train_only_rules.columns)
                else:
                    len_rule = fact
                    min_rule = min(round(col_len*fact),len(X_train_only_rules.columns))

                cols = SelectKBest(k=min_feat_rule).fit(X_train_rules_and_features,y_train).get_feature_names_out()
                cols_rule = SelectKBest(k=min_rule).fit(X_train_only_rules,y_train).get_feature_names_out()

            else:
                 continue

            for algorithm,pipeline in zip(algorithms,pipelines):
                res_rul[(classification_dataset,model + f'_{algorithm}_rules',it,len_rule)] = pipeline(X_train_only_rules[cols_rule], X_test_only_rules[cols_rule], y_train, y_test)
                res_rul[(classification_dataset,model + f'_{algorithm}_rules_and_features',it,fact)] = pipeline(X_train_rules_and_features[cols], X_test_rules_and_features[cols], y_train, y_test)

    for algorithm,pipeline in zip(algorithms,pipelines):
        res_rul[(classification_dataset,algorithm,it,1)] = pipeline(X_train, X_test, y_train, y_test)
    del X_train, X_test

In [ ]:
X

In [ ]:
df

In [ ]:
TARGET_NAME = 'target'
feat_names = [col for col in df.columns if col!=TARGET_NAME]
types = [get_feature_type(df[col], include_binary=True) for col in feat_names]
col = pd.DataFrame(feat_names,types)
num_col = col[col.index == 'continuous']
bin_col = col[col.index == 'binary']
cat_col = col[col.index == 'categorical']
cat_col = cat_col[0].tolist()
dummy_col = pd.get_dummies(data=df, columns=cat_col)
add_col = dummy_col.shape[1] - df.shape[1]

if (add_col < df.shape[0] *0.3) & (dummy_col.shape[1] <  df.shape[0]) & (df.shape[0] < 10000) & (df.shape[1] < 100):
    df = dummy_col
    df.columns = df.columns.str.replace('.','_',regex=True)
    y = df['target']
    X = df.loc[:, df.columns != 'target']
    del df
    rows_data, columns_data = X.shape
    print('Dataset Information')
    print('Rows:',rows_data,)
    print('Columns:',columns_data)
    print('Number of classes:',y.nunique())
    print('Continous columns:', len(num_col))
    print('Binary columns:', len(bin_col))
    print('Categorical columns:',len(cat_col))
    print('-------------------------------------------------')
    else:
        del df
        return pd.DataFrame, pd.DataFrame
    return y, X